In [1]:
!pip install transformers datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.9 MB/s eta 0:00:00


In [3]:
from datasets import load_dataset,DatasetDict
import pandas as pd

Data Loading

In [5]:
data_df=pd.read_csv('/content/labeled_data.csv')
data_df.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [6]:
data_df['tweet']=data_df['tweet'].str.replace('@[a-zA-z0-9]+\s?','',regex=True)
data_df.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT : As a woman you shouldn't complain abo...
1,1,3,0,3,0,1,!!!!! RT : boy dats cold...tyga dwn bad for cu...
2,2,3,0,3,0,1,!!!!!!! RT Dawg!!!! RT : You ever fuck a bitch...
3,3,3,0,2,1,1,!!!!!!!!! RT : she look like a tranny
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT : The shit you hear about me ...


In [8]:
from datasets import Dataset
datasets=Dataset.from_pandas(data_df)

In [9]:
datasets

Dataset({
    features: ['Unnamed: 0', 'count', 'hate_speech', 'offensive_language', 'neither', 'class', 'tweet'],
    num_rows: 24783
})

Creating Train_Test Split

In [18]:
train_data=datasets.train_test_split(test_size=0.25, shuffle=True)
test_data=train_data['test'].train_test_split(shuffle=True)

dataset=DatasetDict({
    'train':train_data['train'],
    'valid':test_data['train'],
    'test':test_data['test']
})

dataset=dataset.remove_columns(['Unnamed: 0', 'count', 'hate_speech', 'offensive_language', 'neither'])
dataset

DatasetDict({
    train: Dataset({
        features: ['class', 'tweet'],
        num_rows: 18587
    })
    valid: Dataset({
        features: ['class', 'tweet'],
        num_rows: 4647
    })
    test: Dataset({
        features: ['class', 'tweet'],
        num_rows: 1549
    })
})

Tokenization Process

In [20]:
from transformers import AutoTokenizer,TFAutoModelForSequenceClassification

In [22]:
model_checkPoint='bert-base-cased'
tokenizer=AutoTokenizer.from_pretrained(model_checkPoint)

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [27]:
text="Hello, Folks Today is Nice Day."
inputs=tokenizer(text)
inputs

{'input_ids': [101, 8667, 117, 12539, 1116, 3570, 1110, 8835, 2295, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [31]:
print(f"Tokenized text: {tokenizer.decode(inputs['input_ids'])}")
print(f"Vocab size is : {tokenizer.vocab_size}")
print(f"Model input features are:{tokenizer.model_input_names}")
print(f"Model max length is : {tokenizer.model_max_length}")


Tokenized text: [CLS] Hello, Folks Today is Nice Day. [SEP]
Vocab size is : 28996
Model input features are:['input_ids', 'token_type_ids', 'attention_mask']
Model max length is : 512


In [26]:
def preprocess_function(example):
  return tokenizer(example['tweet'],padding='max_length',truncation=True)

In [33]:
tokenize_dataset=dataset.map(preprocess_function,batched=True,remove_columns=['tweet'])


Map:   0%|          | 0/18587 [00:00<?, ? examples/s]

Map:   0%|          | 0/4647 [00:00<?, ? examples/s]

Map:   0%|          | 0/1549 [00:00<?, ? examples/s]

In [34]:
train_set=tokenize_dataset['train']
test_set=tokenize_dataset['test']
val_set=tokenize_dataset['valid']

In [39]:
train_set

Dataset({
    features: ['class', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 18587
})

In [40]:
print(train_set[0])

{'class': 1, 'input_ids': [101, 1337, 7979, 1110, 23831, 1228, 1228, 1228, 106, 106, 106, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [45]:
train_feature= {k:train_set[k] for k in tokenizer.model_input_names}
test_feature ={k:test_set[k] for k in tokenizer.model_input_names}
val_feature={k:val_set[k] for k in tokenizer.model_input_names}

In [43]:
import tensorflow as tf

In [44]:
train_tf=tf.data.Dataset.from_tensor_slices((train_feature,train_set['class']))
train_df=train_tf.shuffle(len(train_set)).batch(8)

In [51]:
test_tf=tf.data.Dataset.from_tensor_slices((test_feature,test_set['class']))
test_df=test_tf.shuffle(len(test_set)).batch(8)

In [52]:
val_tf=tf.data.Dataset.from_tensor_slices((val_feature,val_set['class']))
val_df=val_tf.shuffle(len(val_set)).batch(8)

In [50]:
model=TFAutoModelForSequenceClassification.from_pretrained(model_checkPoint,num_labels=3)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [53]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=tf.metrics.SparseCategoricalAccuracy(),
)

In [ ]:
history = model.fit(train_df, validation_data=val_df, epochs=1)

In [ ]:
test_loss, test_acc = model.evaluate(test_df,verbose=2)
print('\nTest accuracy:', test_acc)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
plt.plot(history.history['sparse_categorical_accuracy'])
plt.plot(history.history['val_sparse_categorical_accuracy'])
plt.title('model sparse categorical accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
predict_score_and_class_dict =
{0: 'Hate Speech',
 1: 'Offensive Language',
 2: 'Neither'}

preds = model(tokenizer(["He is useless, I dont know why he came to our neighbourhood", "That guy sucks", "He is such a retard"],return_tensors="tf",padding=True,truncation=True))['logits']

print(preds)

class_preds = np.argmax(preds, axis=1)

for pred in class_preds:
  print(predict_score_and_class_dict[pred])